# Wrangling of WeRateDogs Twitter data

## 1. Data gathering

In [30]:
#import libraries
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import matplotlib.pyplot as plt

### 1.1 Twitter archieve data

In [31]:
#load csv file to tweet_data
tweet_data=pd.read_csv('twitter-archive-enhanced.csv')
tweet_data.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [32]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### 1.2 Image predictions data

In [33]:
# Use requests library to download tsv file from a website
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response=requests.get(url)
with open((url.split('/')[-1]),mode='wb') as file:
    file.write(response.content)

In [34]:
# Read in tsv file as a Pandas DataFrame    
predictions=pd.read_csv('image-predictions.tsv',sep='\t')
predictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [35]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### 1.3 Twitter API data

In [36]:
# Personal API keys, secrets, and tokens have been replaced with placeholders
# consumer_key = '5EnClMecZarmyQA0gDuj8qcyp' 
# consumer_secret = 'xzKWK3h8zJgXL0pgrau3P9Y6nwZYFRUAdWDFmjaJBKDYlSJXUT'
# access_token = '1138148496354304005-CeUuKPVznIPQDpdO7QVNJBzUzX1rh8'
# access_secret = '90vU44jFSvpEOiPtVEstPFTxpnJOUfziHci5AvXE864of'

In [37]:
# Variables created for tweepy query
# auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)

# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [38]:
# write the querying JSON data into tweet_json.txt
# with open('tweet_json.txt','a',encoding = 'utf8') as f:
#     for tweet_id in tweet_data['tweet_id']:
#         try:
#             tweet = api.get_status(tweet_id, tweet_mode = 'extended') # set mode to extended
#             json.dump(tweet._json, f)
#             f.write('\n')
#         except: 
#             print('tweet_id = %s didn\'t work' % (tweet_id))

In [39]:
# Converting JSON data to dataframe
tweets_list =[]
with open('tweet_json.txt') as file:
    for line in file:    
        tweets_dict = {}
        tweets_json = json.loads(line)        
        try:
            tweets_dict['tweet_id'] = tweets_json['id']
        except:
            tweets_dict['tweet_id'] = 'na'

        tweets_dict['retweet_count'] = tweets_json['retweet_count']
        tweets_dict['favorite_count'] = tweets_json['favorite_count']
        
        tweets_list.append(tweets_dict)

In [40]:
tweets_df = pd.DataFrame(tweets_list)

In [41]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332 entries, 0 to 2331
Data columns (total 3 columns):
favorite_count    2332 non-null int64
retweet_count     2332 non-null int64
tweet_id          2332 non-null int64
dtypes: int64(3)
memory usage: 54.7 KB


## 2. Data assessment 

### 2.1 Twitter archieve data

In [42]:
tweet_data

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [43]:
#checking the datatypes and missing values
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [44]:
tweet_data.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [45]:
# number of tweets which has rating denominator not equal to 10
tweet_data.query('rating_denominator != 10').rating_denominator.count()

23

In [46]:
# number of tweets which has rating numerator > 20
tweet_data.query('rating_numerator > 20').rating_numerator.count()

24

In [47]:
#number of null values of exanded urls
tweet_data.expanded_urls.isnull().sum()

59

In [48]:
#number of duplicated tweets
tweet_data.tweet_id.duplicated().sum()

0

In [49]:
tweet_data.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [50]:
tweet_data.name.value_counts()

None         745
a             55
Charlie       12
Cooper        11
Lucy          11
Oliver        11
Lola          10
Penny         10
Tucker        10
Winston        9
Bo             9
Sadie          8
the            8
an             7
Bailey         7
Toby           7
Buddy          7
Daisy          7
Stanley        6
Milo           6
Oscar          6
Bella          6
Dave           6
Rusty          6
Jax            6
Leo            6
Scout          6
Jack           6
Koda           6
Finn           5
            ... 
Kendall        1
Darby          1
Kona           1
Enchilada      1
Maya           1
Callie         1
Juckson        1
Jerome         1
Bowie          1
Tebow          1
Corey          1
Simba          1
Torque         1
Erik           1
Dot            1
Jett           1
Nigel          1
Halo           1
Smiley         1
Caryl          1
Anna           1
Taco           1
Shiloh         1
Snoopy         1
Carbon         1
Schnitzel      1
Ralf           1
Rumpole       

During data assessment the following quality and tidiness issues have been found:
<br>

Data quality issues:
* Remove retweets
* Data completeness: columns 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' should be dropped because they have a lot of missing data and we need only original data for this project.
* Data completeness: 59 tweets which have missing expanded urls - should be removed.
* Data correctness: 23 tweets have rating denominator not equal to 10. These tweets will be removed. 
* Column 'rating_denominator' will be removed because all values are 10. 
* Data correctness: 24 tweets have rating numerator more than 20. These tweets will be removed
* Columns shoud be renamed.
* Timestamp should be datetime format
* Values of source variable should be simplified
    
Tideness issues:
* column headers are values , not variable names. Doggo, puppo, pupper should be values of variable representing the dog stage.
Furthermore, these values should be turned into categorical.



### 2.2 Image predictions data

In [51]:
predictions

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [52]:
predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [53]:
predictions.tweet_id.duplicated().sum()

0

In [54]:
predictions.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


In [55]:
predictions.p1.value_counts()

golden_retriever             150
Labrador_retriever           100
Pembroke                      89
Chihuahua                     83
pug                           57
chow                          44
Samoyed                       43
toy_poodle                    39
Pomeranian                    38
cocker_spaniel                30
malamute                      30
French_bulldog                26
miniature_pinscher            23
Chesapeake_Bay_retriever      23
seat_belt                     22
Staffordshire_bullterrier     20
Siberian_husky                20
German_shepherd               20
Cardigan                      19
web_site                      19
Maltese_dog                   18
Shetland_sheepdog             18
teddy                         18
Eskimo_dog                    18
beagle                        18
Rottweiler                    17
Lakeland_terrier              17
Shih-Tzu                      17
kuvasz                        16
Italian_greyhound             16
          

In [56]:
predictions.p2.value_counts()

Labrador_retriever                104
golden_retriever                   92
Cardigan                           73
Chihuahua                          44
Pomeranian                         42
French_bulldog                     41
Chesapeake_Bay_retriever           41
toy_poodle                         37
cocker_spaniel                     34
Siberian_husky                     33
miniature_poodle                   33
beagle                             28
Eskimo_dog                         27
Pembroke                           27
collie                             27
kuvasz                             26
Italian_greyhound                  22
American_Staffordshire_terrier     21
Pekinese                           21
toy_terrier                        20
Samoyed                            20
chow                               20
malinois                           20
miniature_pinscher                 20
Norwegian_elkhound                 19
Boston_bull                        19
Staffordshir

In [57]:
predictions.p3.value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
Staffordshire_bullterrier         32
chow                              32
beagle                            31
cocker_spaniel                    31
Pomeranian                        29
Pekinese                          29
toy_poodle                        29
Pembroke                          27
Great_Pyrenees                    27
Chesapeake_Bay_retriever          27
malamute                          26
French_bulldog                    26
American_Staffordshire_terrier    24
Cardigan                          23
pug                               23
basenji                           21
toy_terrier                       20
bull_mastiff                      20
Siberian_husky                    19
Boston_bull                       17
Shetland_sheepdog                 17
b

During assessment of image predictions data the following quality and tidiness issues have been found:
<br>

Data quality issues:
* only top 14 prediction values seem to be reasonable. Only data corresponding to the top 14 predictions will be kept
* columns should be renamed: p1 - prediction1, p1_conf- prediction1_CI, p2 - prediction2, p2_conf- prediction2_CI, p3 - prediction3, p3_conf- prediction3_CI. 

Tideness issues:
* A single observation unit (tweet_id) is stored in different tables. tweet_df table should be merged with tweet_data table

### 2.3 Twitter API data

In [58]:
tweets_df

,favorite_count,retweet_count,tweet_id
0,37419,8129,892420643555336193
1,32149,6024,892177421306343426
2,24247,3987,891815181378084864
3,40723,8292,891689557279858688
4,38954,8983,891327558926688256
5,19565,2984,891087950875897856
6,11432,1977,890971913173991426
7,63090,18087,890729181411237888
8,26920,4093,890609185150312448
9,30861,7071,890240255349198849


In [59]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332 entries, 0 to 2331
Data columns (total 3 columns):
favorite_count    2332 non-null int64
retweet_count     2332 non-null int64
tweet_id          2332 non-null int64
dtypes: int64(3)
memory usage: 54.7 KB


In [60]:
tweets_df.tweet_id.value_counts()

749075273010798592    1
671163268581498880    1
743510151680958465    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
700462010979500032    1
780858289093574656    1
699775878809702401    1
880095782870896641    1
760521673607086080    1
776477788987613185    1
691820333922455552    1
715696743237730304    1
714606013974974464    1
760539183865880579    1
813157409116065792    1
833722901757046785    1
805487436403003392    1
842765311967449089    1
670093938074779648    1
756651752796094464    1
674742531037511680    1
670704688707301377    1
667160273090932737    1
674394782723014656    1
672082170312290304    1
759923798737051648    1
685315239903100929    1
818259473185828864    1
                     ..
870308999962521604    1
720775346191278080    1
879492040517615616    1
775733305207554048    1
667911425562669056    1
834209720923721728    1
825026590719483904    1
758405701903519748    1
668986018524233728    1
690938899477221376    1
6788002836490690

In [61]:
#number of duplicated tweets 
tweets_df.tweet_id.duplicated().sum()

0

In [62]:
# number of tweets which has na value
tweets_df.query('tweet_id=="na"').tweet_id.count()

0

During assessment of Twitter API data the following quality issues have been found:
<br>

Data quality issues:
<br>
* Tweet_id should be integer.
* Duplicates of tweets should be removed.
* Tweet_ids which has 'na' value should be dropped.

## 3. Cleaning data

### 3.1 Twitter archieve data

In [63]:
#create a copy of the original data before cleaning
tweet_data_clean=tweet_data.copy()

In [64]:
tweet_data_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [65]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [66]:
#convert timestamp to datetime format
tweet_data_clean.timestamp=pd.to_datetime(tweet_data_clean.timestamp)

In [67]:
#remove retweets
tweet_data_clean=tweet_data_clean[tweet_data_clean.retweeted_status_id.isnull()]

In [68]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2175 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2175 non-null datetime64[ns, UTC]
source                        2175 non-null object
text                          2175 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2117 non-null object
rating_numerator              2175 non-null int64
rating_denominator            2175 non-null int64
name                          2175 non-null object
doggo                         2175 non-null object
floofer                       2175 non-null object
pupper                        2175 non-null object
puppo                         2175 non-null object
dtypes: datetime64[ns, UT

In [69]:
# drop retweet columns
tweet_data_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp'], axis=1, inplace=True)

In [70]:
#remove in reply to status tweets
tweet_data_clean=tweet_data_clean[tweet_data_clean.in_reply_to_status_id.isnull()]

In [71]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 14 columns):
tweet_id                 2097 non-null int64
in_reply_to_status_id    0 non-null float64
in_reply_to_user_id      0 non-null float64
timestamp                2097 non-null datetime64[ns, UTC]
source                   2097 non-null object
text                     2097 non-null object
expanded_urls            2094 non-null object
rating_numerator         2097 non-null int64
rating_denominator       2097 non-null int64
name                     2097 non-null object
doggo                    2097 non-null object
floofer                  2097 non-null object
pupper                   2097 non-null object
puppo                    2097 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(8)
memory usage: 245.7+ KB


In [72]:
#drop in reply to status columns
tweet_data_clean.drop(['in_reply_to_status_id','in_reply_to_user_id'], axis=1, inplace=True)

In [73]:
#removing denominator not equal to 10
tweet_data_clean.query('rating_denominator == 10', inplace=True)

In [74]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2080 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id              2080 non-null int64
timestamp             2080 non-null datetime64[ns, UTC]
source                2080 non-null object
text                  2080 non-null object
expanded_urls         2077 non-null object
rating_numerator      2080 non-null int64
rating_denominator    2080 non-null int64
name                  2080 non-null object
doggo                 2080 non-null object
floofer               2080 non-null object
pupper                2080 non-null object
puppo                 2080 non-null object
dtypes: datetime64[ns, UTC](1), int64(3), object(8)
memory usage: 211.2+ KB


In [75]:
#drop rating denominator column
tweet_data_clean.drop('rating_denominator', axis=1, inplace=True)

In [76]:
tweet_data.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [77]:
#assess columns where rating numerator is more than 20
pd.set_option('display.max_colwidth',-1)
tweet_data[tweet_data.rating_numerator>20].text

188     @dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research                                                                                   
189     @s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10                                    
290     @markhoppus 182/10                                                                                                                                                       
313     @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho                                                                       
340     RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…                             
433     The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd   

In [78]:
#removing numerator which is higher than 20
tweet_data_clean.query('rating_numerator <= 20', inplace=True)

In [79]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 0 to 2355
Data columns (total 11 columns):
tweet_id            2075 non-null int64
timestamp           2075 non-null datetime64[ns, UTC]
source              2075 non-null object
text                2075 non-null object
expanded_urls       2072 non-null object
rating_numerator    2075 non-null int64
name                2075 non-null object
doggo               2075 non-null object
floofer             2075 non-null object
pupper              2075 non-null object
puppo               2075 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(8)
memory usage: 194.5+ KB


In [80]:
# change values of source column
tweet_data_clean.replace('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>','Twitter for iPhone',inplace=True)
tweet_data_clean.replace('<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>','Vine - Make a Scene',inplace=True)
tweet_data_clean.replace('<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>','Twitter Web Client',inplace=True)
tweet_data_clean.replace('<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>','TweetDeck',inplace=True)

In [81]:
#change None values to zeroes
tweet_data_clean.replace('None','',inplace=True)

In [82]:
#create column dog stage which is the combination of 4 columns
tweet_data_clean['dog_stage']=tweet_data_clean.doggo.str.cat(tweet_data_clean.floofer).str.cat(tweet_data_clean.pupper).str.cat(tweet_data_clean.puppo)

In [83]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2075 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id            2075 non-null int64
timestamp           2075 non-null datetime64[ns, UTC]
source              2075 non-null object
text                2075 non-null object
expanded_urls       2072 non-null object
rating_numerator    2075 non-null int64
name                2075 non-null object
doggo               2075 non-null object
floofer             2075 non-null object
pupper              2075 non-null object
puppo               2075 non-null object
dog_stage           2075 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(9)
memory usage: 210.7+ KB


In [84]:
tweet_data_clean.dog_stage.value_counts()

                1740
pupper          220 
doggo           72  
puppo           23  
doggopupper     9   
floofer         9   
doggopuppo      1   
doggofloofer    1   
Name: dog_stage, dtype: int64

In [85]:
# drop values that are not pupper, doggo, puppo or floofer
tweet_data_clean.query('dog_stage in ["","pupper","doggo","puppo","floofer"]', inplace=True)

In [86]:
tweet_data_clean.dog_stage.value_counts()

           1740
pupper     220 
doggo      72  
puppo      23  
floofer    9   
Name: dog_stage, dtype: int64

In [87]:
tweet_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2064 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id            2064 non-null int64
timestamp           2064 non-null datetime64[ns, UTC]
source              2064 non-null object
text                2064 non-null object
expanded_urls       2061 non-null object
rating_numerator    2064 non-null int64
name                2064 non-null object
doggo               2064 non-null object
floofer             2064 non-null object
pupper              2064 non-null object
puppo               2064 non-null object
dog_stage           2064 non-null object
dtypes: datetime64[ns, UTC](1), int64(2), object(9)
memory usage: 209.6+ KB


In [88]:
# drop columns pupper, doggo, puppo and floofer
tweet_data_clean.drop(['doggo','floofer','pupper','puppo'], axis=1, inplace=True)

In [89]:
tweet_data_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,name,dog_stage
0,892420643555336193,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,Phineas,
1,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,Tilly,
2,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,Archie,
3,891689557279858688,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,Darla,
4,891327558926688256,2017-07-29 16:00:24+00:00,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,Franklin,


In [90]:
# convert dog_stage to categorical data
tweet_data_clean.dog_stage=tweet_data_clean.dog_stage.astype('category')

In [91]:
#rename columns
tweet_data_clean.rename(columns={'timestamp': 'tweet_timestamp', 'text': 'tweet_text', 'rating_numerator': 'rating_out_of_10','name': 'dog_name', 'source': 'tweet_source'}, inplace=True)

### 3.2 Image predictions data

In [92]:
#create copy of data before cleaning
predictions_clean=predictions.copy()

In [93]:
predictions_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [94]:
predictions_clean.rename(columns={'p1': 'prediction1', 'p1_conf': 'prediction1_CI', 'p2': 'prediction2','p2_conf': 'prediction2_CI', 'p3': 'prediction3','p3_conf':'prediction3_CI','p1_dog':'prediction1_dog','p2_dog':'prediction2_dog','p3_dog':'prediction3_dog'}, inplace=True)

In [95]:
#extract top 14 predictions
top_14_dogs=predictions_clean.prediction1.value_counts().iloc[:14].index.tolist()

In [96]:
top_14_dogs

['golden_retriever',
 'Labrador_retriever',
 'Pembroke',
 'Chihuahua',
 'pug',
 'chow',
 'Samoyed',
 'toy_poodle',
 'Pomeranian',
 'cocker_spaniel',
 'malamute',
 'French_bulldog',
 'miniature_pinscher',
 'Chesapeake_Bay_retriever']

In [97]:
#keep only rows which correspond to top 14 predictions
predictions_clean=predictions_clean[predictions_clean.prediction1.isin(top_14_dogs)]

In [98]:
predictions_clean.prediction1.value_counts()

golden_retriever            150
Labrador_retriever          100
Pembroke                    89 
Chihuahua                   83 
pug                         57 
chow                        44 
Samoyed                     43 
toy_poodle                  39 
Pomeranian                  38 
malamute                    30 
cocker_spaniel              30 
French_bulldog              26 
Chesapeake_Bay_retriever    23 
miniature_pinscher          23 
Name: prediction1, dtype: int64

In [99]:
predictions_clean.head()

,tweet_id,jpg_url,img_num,prediction1,prediction1_CI,prediction1_dog,prediction2,prediction2_CI,prediction2_dog,prediction3,prediction3_CI,prediction3_dog
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,golden_retriever,0.775930,True,Tibetan_mastiff,0.093718,True,Labrador_retriever,0.072427,True
13,666082916733198337,https://pbs.twimg.com/media/CT5m4VGWEAAtKc8.jpg,1,pug,0.489814,True,bull_mastiff,0.404722,True,French_bulldog,0.048960,True
23,666345417576210432,https://pbs.twimg.com/media/CT9Vn7PWoAA_ZCM.jpg,1,golden_retriever,0.858744,True,Chesapeake_Bay_retriever,0.054787,True,Labrador_retriever,0.014241,True


In [108]:
predictions_clean_1=predictions_clean[['tweet_id','prediction1','prediction1_CI','prediction1_dog']].copy()
predictions_clean_1

,tweet_id,prediction1,prediction1_CI,prediction1_dog
4,666049248165822465,miniature_pinscher,0.560311,True
7,666055525042405380,chow,0.692517,True
10,666063827256086533,golden_retriever,0.775930,True
13,666082916733198337,pug,0.489814,True
23,666345417576210432,golden_retriever,0.858744,True
24,666353288456101888,malamute,0.336874,True
27,666396247373291520,Chihuahua,0.978108,True
32,666428276349472768,Pembroke,0.371361,True
34,666435652385423360,Chesapeake_Bay_retriever,0.184130,True
35,666437273139982337,Chihuahua,0.671853,True


### 3.3 Twitter API data

In [100]:
# create a copy of the dataset before cleaning
tweets_df_clean=tweets_df.copy()

In [101]:
#drop tweet_id which nas value 'na'
tweets_df_clean.query('tweet_id != "na"', inplace=True)
tweets_df_clean.query('tweet_id != "nan"', inplace=True)
tweets_df_clean.query('tweet_id != "NA"', inplace=True)
tweets_df_clean.query('tweet_id != "inf"', inplace=True)

In [103]:
#drop duplicate tweet_id rows
tweets_df_clean.tweet_id.drop_duplicates(inplace=True)

In [106]:
#merging with tweet_data_clean table on tweet_id

twitter_archive_master=tweet_data_clean.merge(tweets_df_clean, on='tweet_id')

In [107]:
twitter_archive_master.head()

,tweet_id,tweet_timestamp,tweet_source,tweet_text,expanded_urls,rating_out_of_10,dog_name,dog_stage,favorite_count,retweet_count
0,892420643555336193,2017-08-01 16:23:56+00:00,Twitter for iPhone,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,Phineas,,37419,8129
1,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,Tilly,,32149,6024
2,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,Archie,,24247,3987
3,891689557279858688,2017-07-30 15:58:51+00:00,Twitter for iPhone,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,Darla,,40723,8292
4,891327558926688256,2017-07-29 16:00:24+00:00,Twitter for iPhone,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,Franklin,,38954,8983


In [110]:
twitter_archive_master=twitter_archive_master.merge(predictions_clean_1, on='tweet_id')

In [111]:
twitter_archive_master.head()

,tweet_id,tweet_timestamp,tweet_source,tweet_text,expanded_urls,rating_out_of_10,dog_name,dog_stage,favorite_count,retweet_count,prediction1,prediction1_CI,prediction1_dog
0,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,Tilly,,32149,6024,Chihuahua,0.323581,True
1,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,Archie,,24247,3987,Chihuahua,0.716012,True
2,891087950875897856,2017-07-29 00:08:17+00:00,Twitter for iPhone,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,,,19565,2984,Chesapeake_Bay_retriever,0.425595,True
3,890729181411237888,2017-07-28 00:22:40+00:00,Twitter for iPhone,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,,,63090,18087,Pomeranian,0.566142,True
4,890240255349198849,2017-07-26 15:59:51+00:00,Twitter for iPhone,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,https://twitter.com/dog_rates/status/890240255349198849/photo/1,14,Cassie,doggo,30861,7071,Pembroke,0.511319,True


## 4. Storing data

In [115]:
twitter_archive_master.to_csv('twitter_archive_master.csv',index=False, encoding='utf-8')

In [116]:
df=pd.read_csv('twitter_archive_master.csv')
df.head()

,tweet_id,tweet_timestamp,tweet_source,tweet_text,expanded_urls,rating_out_of_10,dog_name,dog_stage,favorite_count,retweet_count,prediction1,prediction1_CI,prediction1_dog
0,892177421306343426,2017-08-01 00:17:27+00:00,Twitter for iPhone,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,Tilly,NaN,32149,6024,Chihuahua,0.323581,True
1,891815181378084864,2017-07-31 00:18:03+00:00,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,Archie,NaN,24247,3987,Chihuahua,0.716012,True
2,891087950875897856,2017-07-29 00:08:17+00:00,Twitter for iPhone,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,NaN,NaN,19565,2984,Chesapeake_Bay_retriever,0.425595,True
3,890729181411237888,2017-07-28 00:22:40+00:00,Twitter for iPhone,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,NaN,NaN,63090,18087,Pomeranian,0.566142,True
4,890240255349198849,2017-07-26 15:59:51+00:00,Twitter for iPhone,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,https://twitter.com/dog_rates/status/890240255349198849/photo/1,14,Cassie,doggo,30861,7071,Pembroke,0.511319,True


## 4. Visualising data